In [5]:
import torch

from utils import load_clip, extract_features, get_text_embeds
from sae import train_autoencoder
import clip

In [6]:
models = {
    "resnet50": "RN50",
    "vit_b16": "ViT-B/16",
    "vit_l14": "ViT-L/14"
}

clip_model_name = models["resnet50"]
device = "cuda" if torch.cuda.is_available() else "cpu"
#//////////////////////////////

In [7]:
clip_model, preprocess = load_clip(clip_model_name, device)

In [4]:
# Specify the path to your CSV file that contains the local image paths.
csv_file = "cc3m_testing.csv"  

print("Extracting image features from local dataset...")
# This function applies the preprocessing and extracts features.
processed_dataset = extract_features(csv_file, clip_model, preprocess, device)
print("Image feature extraction complete.")
print(f"Processed dataset contains {len(processed_dataset)} samples.")

Extracting image features from local dataset...


Generating train split: 10 examples [00:00, 1549.60 examples/s]
Saving the dataset (1/1 shards): 100%|██████████| 10/10 [00:00<00:00, 490.93 examples/s]

Image feature extraction complete.
Processed dataset contains 10 samples.


In [ ]:
print("Training sparse autoencoder...")
neurons_fired = train_autoencoder(clip_model_name)
print(f"Training complete. Neurons fired: {neurons_fired}")

In [4]:
# %% [code]
# Specify the path to your text file with one text (or caption) per line.
text_file = "20k_vocab.txt"

text_embeddings = get_text_embeds(clip_model, text_file, device)
print("Extracted text embeddings.")
print("Text embeddings shape:", text_embeddings.shape)


Saving the dataset (1/1 shards): 100%|██████████| 20000/20000 [00:01<00:00, 11574.64 examples/s]


Extracted text embeddings.
Text embeddings shape: torch.Size([20000, 1024])


In [1]:
from sparse_autoencoder import (
    ActivationResamplerHyperparameters,
    AutoencoderHyperparameters,
    Hyperparameters,
    LossHyperparameters,
    Method,
    OptimizerHyperparameters,
    Parameter,
    Pipeline,
    PipelineHyperparameters,
    SourceDataHyperparameters,
    SourceModelHyperparameters,
    SweepConfig,
    sweep,
)

from sparse_autoencoder.loss.abstract_loss import AbstractLoss

/home/smusa28/miniconda3/envs/eecs6322-rpc/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
sweep_config = SweepConfig(
    parameters=Hyperparameters(
        loss=LossHyperparameters(
            l1_coefficient=Parameter(values=[3e-5, 1.5e-4, 3e-4, 1.5e-3, 3e-3]),
        ),
        optimizer=OptimizerHyperparameters(
            lr=Parameter(values=[1e-5, 5e-5, 1e-4, 5e-4, 1e-3]),
        ),
        source_model=SourceModelHyperparameters(
            # name=Parameter("openai/clip"), # idk if i should specify the model
            # cache_names=Parameter(["vision_model.encoder.layers.11"]),  # Extract from last layer
            hook_dimension=Parameter(768 if clip_model_name == "ViT-B/16" else 1024)
        ),
        source_data=SourceDataHyperparameters(
            dataset_path=Parameter("cc3m_clip_features"),  # CC3M dataset
            context_size=Parameter(256),  # Number of tokens/images to process per batch
            pre_tokenized=Parameter(value=False),  # CC3M is not pre-tokenized
            pre_download=Parameter(value=False),  # Stream instead of downloading
            tokenizer_name=Parameter("openai/clip-vit-base-patch32")
        ),
        autoencoder=AutoencoderHyperparameters(
            expansion_factor=Parameter(values=[2,4,8])
        ),
        # num_epochs = Parameter(200),
        # resample_interval = Parameter(10)
    ),
    method=Method.RANDOM,
)

TypeError: SourceModelHyperparameters.__init__() missing 2 required positional arguments: 'name' and 'cache_names'

In [ ]:
pipeline = Pipeline(sweep_config)

num_neurons_fired = pipeline.train_autoencoder()

In [2]:

import os
import math
import datetime
from pathlib import Path
import torch
import numpy as np
from time import time

# Import necessary components from the sparse autoencoder repository.
# (Adjust these imports if your installation or repository structure is different.)
from sparse_autoencoder import (
    ActivationResampler,
    AdamWithReset,
    L2ReconstructionLoss,
    LearnedActivationsL1Loss,
    LossReducer,
    SparseAutoencoder,
    Pipeline,
)
from datasets import load_from_disk

In [3]:

# --------- CONFIGURATION (edit these values as needed) ---------
# Device configuration:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cpu


In [5]:
# Load the precomputed CC3M CLIP embeddings dataset.
embeddings_dataset_path = "cc3m_clip_features"  # folder where embeddings were saved
print(f"Loading CC3M embeddings from {embeddings_dataset_path} ...")
source_dataset = load_from_disk(embeddings_dataset_path)
print(f"Loaded dataset with {len(source_dataset)} samples.")


Loading CC3M embeddings from cc3m_clip_features ...
Loaded dataset with 10 samples.


In [6]:
# Set the input dimension for the autoencoder.
# For example, if you used a CLIP RN50 model, the embedding dimension is likely 1024.
autoencoder_input_dim = 1024  
expansion_factor = 16  # You can adjust this factor
n_learned_features = int(autoencoder_input_dim * expansion_factor)
n_components = 1  # If you only have one hook/activation source; adjust if you use multiple layers

In [7]:
# --------- Build the Sparse Autoencoder ---------
print("Creating the Sparse Autoencoder ...")
autoencoder = SparseAutoencoder(
    n_input_features=autoencoder_input_dim,
    n_learned_features=n_learned_features,
    n_components=n_components,
).to(device)
print("Autoencoder created.")

Creating the Sparse Autoencoder ...


TypeError: SparseAutoencoder.__init__() got an unexpected keyword argument 'n_input_features'

In [ ]:







# --------- Set up the Loss and Optimizer ---------
# We combine an L1 loss on the learned activations (for sparsity) with an L2 reconstruction loss.
l1_coeff = 1.5e-3  # Adjust as needed
loss = LossReducer(
    LearnedActivationsL1Loss(l1_coefficient=float(l1_coeff)),
    L2ReconstructionLoss(),
)
print("Loss function created.")

# Set up the custom Adam optimizer.
lr = 5e-4  # Choose an appropriate learning rate
optimizer = AdamWithReset(
    params=autoencoder.parameters(),
    named_parameters=autoencoder.named_parameters(),
    lr=float(lr),
    betas=(0.9, 0.999),
    eps=1e-8,
    weight_decay=0,
    has_components_dim=True,
)
print("Optimizer created.")

# --------- Set up the Activation Resampler ---------
# This component periodically resamples activations during training.
activation_resampler = ActivationResampler(
    resample_interval=10,  # e.g., every 10 iterations/epochs (adjust as needed)
    n_activations_activity_collate=10,
    max_n_resamples=math.inf,
    n_learned_features=n_learned_features,
    resample_epoch_freq=1,  # Resample every epoch, adjust if necessary
    resample_dataset_size=1000,  # Use a subset for resampling; adjust according to dataset size.
)
print("Activation resampler created.")

# --------- Set up the Pipeline ---------
# Define a directory for saving checkpoints.
checkpoint_directory = Path("./sae_checkpoints")
checkpoint_directory.mkdir(exist_ok=True)
print(f"Checkpoints will be saved in {checkpoint_directory}")

# Create the training pipeline.
pipeline = Pipeline(
    activation_resampler=activation_resampler,
    autoencoder=autoencoder,
    checkpoint_directory=checkpoint_directory,
    loss=loss,
    optimizer=optimizer,
    device=device,
    args=None,  # No additional args passed in this minimal example.
)
print("Pipeline created.")

# --------- Run Training ---------
num_epochs = 200  # Set the number of training epochs
train_batch_size = 256  # Adjust according to memory and dataset size

print("Starting training ...")
start_time = time()

# For this minimal example, we assume the Pipeline class has a simple training method 
# that accepts the source dataset, batch size, and number of epochs.
pipeline.train_autoencoder(
    source_dataset=source_dataset,
    num_epochs=num_epochs,
    train_batch_size=train_batch_size,
)

total_time = time() - start_time
print(f"Training complete in {total_time:.2f} seconds.")
